# Forschungsbericht: Mehrsprachiger Konversationstravel-Assistent

Dieser Bericht dokumentiert die Konzeption, Implementierung und iterative Verbesserung eines mehrsprachigen Konversationstravel-Assistenten. Der Assistent ist in der Lage, Reiseanfragen sowohl auf Englisch als auch auf Deutsch zu verstehen und zu beantworten und extrahiert strukturierte Informationen (Slots) für nachgelagerte Dienste wie Wetter-, Flug- und Hotel-APIs.

## 1. Projektübersicht

Ziel des Projekts ist es, einen robusten Chatbot zu entwickeln, der:
- Freiform-Benutzerfragen zu Reisen (Wetter, Flüge, Hotels) akzeptiert.
- Intention und relevante Slots (z.B. Stadt, Daten, Herkunft, Ziel) extrahiert.
- Sowohl englische als auch deutsche Anfragen unterstützt.
- Mit realen APIs für Wetter, Flüge und Hotels integriert ist.
- Fallback-Mechanismen und Normalisierung für unklare oder unvollständige Benutzereingaben bietet.

## 2. Datensammlung und Annotation

### 2.1. Ursprüngliche Datenquellen
- Englische und deutsche Reiseanfragen wurden gesammelt und decken verschiedene Szenarien ab (Einzeldatum, Datumsbereich, Stadt, Herkunft/Ziel).
- Bestehende Datensätze wie GermEval und synthetische Daten wurden als Basis verwendet.

### 2.2. Erweiterung der deutschen Daten
- Zusätzliche deutsche Anfragen wurden manuell erstellt und annotiert, um folgende Bereiche abzudecken:
  - Flugbuchung (mit und ohne Datumsbereich)
  - Hotelbuchung (mit Erwachsenen/Kinder, Frühstück etc.)
  - Wetteranfragen (Einzeldatum, Datumsbereich, Monat)
- Beispielannotation:
  ```
  {
    "text": "Ich will einen Flug von Hamburg nach Zürich am 12. September",
    "slots": [
      {"slot": "origin", "value": "Hamburg"},
      {"slot": "destination", "value": "Zürich"},
      {"slot": "date", "value": "12. September"}
    ]
  }
  ```

### 2.3. Zusammenführen der Datensätze
- Alle annotierten Daten wurden mit einem eigenen Skript zu einem umfassenden, mehrsprachigen Slot-NER-Datensatz zusammengeführt.

## 3. Modelltraining

### 3.1. Slot-NER-Modell
- Ein Transformer-basiertes Modell (z.B. BERT) wurde für die Slot-Extraktion trainiert.
- Das Modell wurde nach jeder Datenerweiterung erneut trainiert und sowohl mit englischen als auch deutschen Anfragen validiert.
- Das Trainingsskript unterstützt sowohl Einzeldatum- als auch Datumsbereich-Annotationen.

### 3.2. Intent-Klassifikation
- Ein separater Intent-Klassifikator wurde trainiert, um zwischen Wetter-, Flug-, Hotel- und anderen Anfragen zu unterscheiden.

### 3.3. Evaluation
- Die Modelle wurden mit Genauigkeit und F1-Score auf einem separaten Validierungsdatensatz bewertet.
- Manuelle und automatisierte Tests wurden mit einer Vielzahl von Anfragen in beiden Sprachen durchgeführt.

## 4. Chatbot-Logik und Slot-Normalisierung

### 4.1. Kernpipeline
- Die Benutzereingabe wird zunächst auf die Sprache geprüft und bei Bedarf übersetzt.
- Intention und Slots werden mit den trainierten Modellen extrahiert.
- Die Slot-Normalisierungslogik stellt sicher:
  - Wenn nur `date` vorhanden ist, wird dieses sowohl als `start_date` als auch als `end_date` verwendet.
  - Wenn nur eines von `start_date` oder `end_date` vorhanden ist, wird es als Einzeldatum behandelt.
  - Fallback-Regex wird für Flugabfragen verwendet, um `origin` und `destination` zu extrahieren, falls das Modell sie nicht erkennt.
  - Deutsche Monatsnamen werden für die Weiterverarbeitung ins Englische normalisiert.

### 4.2. API-Integration
- Wetter-, Flug- und Hotelinformationen werden über reale APIs (OpenWeatherMap, Google Flights, Booking.com) abgerufen.
- Städtenamen werden für Flugabfragen in IATA-Codes umgewandelt.

### 4.3. Debugging und Tests
- Umfangreiche Debug-Ausgaben für Slot-Extraktion und Normalisierung.
- Eine Testsuite mit über 20 Szenarien (Englisch und Deutsch) validiert die End-to-End-Pipeline.

## 5. Iterative Verbesserung

### 5.1. Fehleranalyse
- Testergebnisse wurden analysiert, um Fehlerfälle zu identifizieren (z.B. fehlende Slots, mehrdeutige Daten).
- Die Trainingsdaten wurden daraufhin erweitert und die Normalisierungslogik verbessert.

### 5.2. Robustheit
- Der Chatbot ist dank Normalisierung und Fallback-Logik robust gegenüber unvollständigen oder mehrdeutigen Eingaben.
- Mehrsprachigkeit wird durch Übersetzung und direkte Annotation erreicht.

## 6. Letzter Schritt: Der Bot in Aktion

Der finale Chatbot ist in der Lage:
- Reiseanfragen sowohl auf Englisch als auch auf Deutsch zu verstehen und zu beantworten.
- Strukturierte Informationen für nachgelagerte APIs zu extrahieren.
- Eine Vielzahl von Datumsformaten und Slot-Mehrdeutigkeiten zu verarbeiten.
- Echtzeit-Reiseberatung zu bieten, einschließlich Wetter-, Flug- und Hotelinformationen.
- **Neu:** Automatische Integration von aktuellen Empfehlungen zu Restaurants und Sehenswürdigkeiten aus TripAdvisor (über die Travel Advisor RapidAPI). Für jede Benutzeranfrage werden die besten Restaurants und Attraktionen am Zielort direkt im Chatbot-Antworttext angezeigt.

**Beispielinteraktion:**

- **Benutzer:** "Finde Flüge von Berlin nach Rom vom 1. Juli bis 10. Juli"
- **Bot:** Gibt Flugoptionen, Wetter für beide Städte, Hotelvorschläge und aktuelle TripAdvisor-Empfehlungen (z.B. Top-Restaurants und Sehenswürdigkeiten in Rom) zurück.

**Testen:**  
Eine umfassende Testsuite stellt sicher, dass der Bot sowohl bei englischen als auch bei deutschen Anfragen zuverlässig funktioniert. Debug-Ausgaben unterstützen die weitere Verbesserung.

## 7. Fazit

Dieses Projekt zeigt einen praxisnahen Ansatz zum Aufbau eines mehrsprachigen, robusten Konversationstravel-Assistenten. Erfolgsfaktoren sind:
- Iterative Datenerweiterung und Annotation.
- Sorgfältige Slot-Normalisierung und Fallback-Logik.
- Integration mit realen APIs.
- Umfassende Tests und Debugging.

Zukünftige Arbeiten könnten beinhalten:
- Erweiterung auf weitere Sprachen.
- Verbesserung der NER-bewussten Übersetzung.
- Erweiterte Kontextbehandlung und Dialogmanagement.